In [1]:
API_KEY ="sk-b56c488f33b94df297a6314bd037b805"

In [2]:
import nest_asyncio
nest_asyncio.apply()
import aiohttp
import asyncio
import pandas as pd
import json
import os
import re
import ast
from datetime import datetime
from tqdm import tqdm
from tqdm.asyncio import tqdm as atqdm

CHECKPOINT_FILE = "conversion_checkpoint.json"
PARTIAL_SIMPLIFIED_FILE = "simplified_partial.csv"
PARTIAL_TRADITIONAL_FILE = "traditional_partial.csv"

def load_checkpoint():
    """載入處理進度"""
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, "r", encoding="utf-8") as f:
            ckpt = json.load(f)
        print(f"🔄 偵測到進度檔，將從 index {ckpt['last_index'] + 1} 繼續")
        return ckpt["last_index"], ckpt.get("processed_indices", [])
    return -1, []

def save_checkpoint(last_index, processed_indices):
    """儲存處理進度"""
    with open(CHECKPOINT_FILE, "w", encoding="utf-8") as f:
        json.dump({
            "last_index": last_index,
            "processed_indices": processed_indices
        }, f)


# 🎯 簡體轉繁體系統 - 使用 API 進行轉換
print("🚀 啟動簡體轉繁體轉換系統...")

# 定義大陸用語到台灣用語的對照表
mainland_to_taiwan_terms = {
    "計算機": "電腦", "軟件": "軟體", "硬件": "硬體", "網絡": "網路", 
    "數據": "資料", "程序": "程式", "信息": "資訊", "出租車": "計程車",
    "公交車": "公車", "地鐵": "捷運", "質量": "品質", "服務員": "服務生",
    "土豆": "馬鈴薯", "西紅柿": "番茄", "鼠標": "滑鼠", "打印機": "印表機",
    "文件夾": "資料夾", "博客": "部落格", "視頻": "影片", "音頻": "音訊",
    "激活": "啟用", "默認": "預設", "註冊": "註冊", "登錄": "登入"
}

async def convert_to_traditional_api_async(text, session, model_endpoint, api_key, model_name):
    """使用 API 非同步將簡體大陸用語轉換成台灣繁體用語"""
    
    prompt = f"""請將以下簡體中文文本轉換為台灣繁體中文。

要求：
1. 將所有簡體字轉換為繁體字
2. 將大陸用語轉換為台灣常用詞彙（例如：計算機→電腦、軟件→軟體、網絡→網路等）
3. 保持句子結構和語意不變
4. 只回傳轉換後的繁體中文文本，不要有任何其他說明

原文：{text}

繁體中文："""

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "max_tokens": 500
    }

    try:
        async with session.post(model_endpoint, headers=headers, json=payload, timeout=90) as response:
            data = await response.json()
            reply = data.get("choices", [{}])[0].get("message", {}).get("content", None)
            
            # 清理回應，移除可能的前綴說明
            if reply:
                reply = reply.strip()
                # 移除可能的前綴如 "繁體中文：" 等
                prefixes = ["繁體中文：", "繁體版本：", "轉換結果：", "台灣繁體："]
                for prefix in prefixes:
                    if reply.startswith(prefix):
                        reply = reply[len(prefix):].strip()
            
            return reply if reply else text  # 如果轉換失敗，返回原文
    except Exception as e:
        print(f"⚠️  API 錯誤: {e}")
        return None


def basic_simplified_to_traditional(text):
    """基本的簡繁轉換（備用方案）"""
    # 使用詞彙對照表進行基本轉換
    result = text
    for simp, trad in mainland_to_taiwan_terms.items():
        result = result.replace(simp, trad)
    return result


async def process_dataset_async_batched(df, model_endpoint, api_key, model_name="gemma-3-27b-it",
                                        text_col='text', batch_size=10):
    """
    非同步批次處理資料集，將簡體轉繁體
    """
    print(f"📊 處理資料集：{len(df)} 筆")
    
    last_index, processed_indices = load_checkpoint()
    processed_set = set(processed_indices)
    
    simplified_results = []
    traditional_results = []

    # 載入已處理的部分結果
    if os.path.exists(PARTIAL_SIMPLIFIED_FILE):
        simp_df = pd.read_csv(PARTIAL_SIMPLIFIED_FILE)
        simplified_results = simp_df.to_dict("records")
        print(f"🔁 已讀取簡體資料結果：{len(simplified_results)} 筆")
    
    if os.path.exists(PARTIAL_TRADITIONAL_FILE):
        trad_df = pd.read_csv(PARTIAL_TRADITIONAL_FILE)
        traditional_results = trad_df.to_dict("records")
        print(f"🔁 已讀取繁體資料結果：{len(traditional_results)} 筆")

    texts = df[text_col].tolist()
    indices = df.index.tolist()
    
    # 計算總批次數和需要處理的資料數
    total_batches = (len(texts) + batch_size - 1) // batch_size
    total_to_process = len([idx for idx in indices if idx not in processed_set])
    
    print(f"📋 總批次數: {total_batches}")
    print(f"📋 待處理資料: {total_to_process} 筆")
    print(f"📋 已完成資料: {len(processed_set)} 筆")

    async with aiohttp.ClientSession() as session:
        # 使用 tqdm 創建主進度條
        with tqdm(total=total_to_process, desc="🔄 總體轉換進度", unit="筆") as pbar:
            for batch_start in range(0, len(texts), batch_size):
                batch_texts = texts[batch_start:batch_start+batch_size]
                batch_indices = indices[batch_start:batch_start+batch_size]

                # 跳過已處理的索引
                filtered_batch = [
                    (idx, text) for idx, text in zip(batch_indices, batch_texts)
                    if idx not in processed_set
                ]
                
                if not filtered_batch:
                    continue

                filtered_indices, filtered_texts = zip(*filtered_batch) if filtered_batch else ([], [])
                
                # 顯示當前批次資訊
                batch_num = (batch_start // batch_size) + 1
                pbar.set_description(f"🔄 批次 {batch_num}/{total_batches} | 處理中")

                # 執行 API 轉換
                tasks = [
                    convert_to_traditional_api_async(text, session, model_endpoint, api_key, model_name)
                    for text in filtered_texts
                ]
                responses = await asyncio.gather(*tasks)

                for i, response in enumerate(responses):
                    idx = filtered_indices[i]
                    original_text = filtered_texts[i]
                    traditional_text = response if response else basic_simplified_to_traditional(original_text)
                    
                    # 獲取原始資料的其他欄位
                    original_row = df.iloc[idx]
                    
                    # 簡體資料（原始資料）
                    simp_record = {
                        'text': original_text,
                        'text_length': len(original_text),
                        'language_type': '簡體中文'
                    }
                    
                    # 繁體資料（轉換後）
                    trad_record = {
                        'text': traditional_text,
                        'text_length': len(traditional_text),
                        'language_type': '繁體中文',
                        'original_text': original_text  # 保留原文以便對照
                    }
                    
                    # 複製其他欄位
                    for col in df.columns:
                        if col != text_col and col in original_row:
                            simp_record[col] = original_row[col]
                            trad_record[col] = original_row[col]
                    
                    simplified_results.append(simp_record)
                    traditional_results.append(trad_record)
                    processed_set.add(idx)
                    
                    # 更新進度條
                    pbar.update(1)
                
                # 定期保存進度
                if len(filtered_batch) > 0:
                    pd.DataFrame(simplified_results).to_csv(PARTIAL_SIMPLIFIED_FILE, index=False, encoding="utf-8-sig")
                    pd.DataFrame(traditional_results).to_csv(PARTIAL_TRADITIONAL_FILE, index=False, encoding="utf-8-sig")
                    save_checkpoint(max(processed_set), list(processed_set))
                    
                    # 更新進度條的後綴資訊
                    completion_rate = (len(processed_set) / len(df)) * 100
                    pbar.set_postfix({
                        '完成率': f'{completion_rate:.1f}%',
                        '已轉換': len(processed_set)
                    })

    return simplified_results, traditional_results


def save_final_results(simplified_results, traditional_results):
    """儲存最終結果"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print("\n💾 正在儲存檔案...")
    
    # 儲存簡體中文資料
    simp_df = pd.DataFrame(simplified_results)
    simp_csv = f"simplified_chinese_data_{timestamp}.csv"
    simp_json = f"simplified_chinese_data_{timestamp}.json"
    
    with tqdm(total=2, desc="📝 儲存簡體中文資料") as pbar:
        simp_df.to_csv(simp_csv, index=False, encoding='utf-8-sig')
        pbar.update(1)
        simp_df.to_json(simp_json, orient='records', force_ascii=False, indent=2)
        pbar.update(1)
    
    # 儲存繁體中文資料
    trad_df = pd.DataFrame(traditional_results)
    trad_csv = f"traditional_chinese_data_{timestamp}.csv"
    trad_json = f"traditional_chinese_data_{timestamp}.json"
    
    with tqdm(total=2, desc="📝 儲存繁體中文資料") as pbar:
        trad_df.to_csv(trad_csv, index=False, encoding='utf-8-sig')
        pbar.update(1)
        trad_df.to_json(trad_json, orient='records', force_ascii=False, indent=2)
        pbar.update(1)
    
    # 儲存合併資料集
    combined_df = pd.concat([simp_df, trad_df], ignore_index=True)
    combined_csv = f"combined_chinese_data_{timestamp}.csv"
    
    with tqdm(total=1, desc="📝 儲存合併資料集") as pbar:
        combined_df.to_csv(combined_csv, index=False, encoding='utf-8-sig')
        pbar.update(1)
    
    print(f"\n✅ 儲存完成:")
    print(f"  📄 簡體中文資料: {simp_csv} ({len(simp_df)} 筆)")
    print(f"  📄 繁體中文資料: {trad_csv} ({len(trad_df)} 筆)")
    print(f"  📄 合併資料集: {combined_csv} ({len(combined_df)} 筆)")
    print(f"  📋 JSON格式也已儲存")
    
    return simp_df, trad_df, combined_df


# 主要執行流程
print("="*60)

# 載入目標資料集
data_path = " LLM_Aug_102925.csv"  # 注意檔名前有空格
print(f"📂 載入資料集: {data_path}")

try:
    with tqdm(total=1, desc="📥 讀取資料集") as pbar:
        source_df = pd.read_csv(data_path)
        pbar.update(1)
    
    print(f"✅ 成功載入資料集，共 {len(source_df)} 筆記錄")
    print(f"📋 欄位: {source_df.columns.tolist()}")
    
    # 設定 API 參數
    MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"
    OPENWEBUI_API_KEY = API_KEY
    MODEL_NAME = "gemma-3-27b-it"
    BATCH_SIZE = 10  # 每批處理數量
    
    print(f"\n🚀 開始非同步批次轉換")
    print(f"🤖 使用模型: {MODEL_NAME}")
    print(f"📦 批次大小: {BATCH_SIZE} 筆/批次")
    print(f"⏱️  預估批次數: {(len(source_df) + BATCH_SIZE - 1) // BATCH_SIZE}")
    print("="*60)
    
    # 執行轉換
    simplified_results, traditional_results = await process_dataset_async_batched(
        df=source_df,
        model_endpoint=MODEL_API_ENDPOINT,
        api_key=OPENWEBUI_API_KEY,
        model_name=MODEL_NAME,
        text_col='text',
        batch_size=BATCH_SIZE
    )
    
    # 統計結果
    print(f"\n" + "="*60)
    print(f"📊 轉換結果統計:")
    print(f"  📝 簡體中文資料: {len(simplified_results)} 筆")
    print(f"  📝 繁體中文資料: {len(traditional_results)} 筆")
    print(f"  📈 總資料量: {len(simplified_results) + len(traditional_results)} 筆")
    
    # 顯示轉換範例
    if traditional_results:
        print(f"\n🔄 轉換範例:")
        for i in range(min(3, len(traditional_results))):
            simp_preview = simplified_results[i]['text'][:40] + "..." if len(simplified_results[i]['text']) > 40 else simplified_results[i]['text']
            trad_preview = traditional_results[i]['text'][:40] + "..." if len(traditional_results[i]['text']) > 40 else traditional_results[i]['text']
            print(f"  {i+1}.")
            print(f"     簡體: {simp_preview}")
            print(f"     繁體: {trad_preview}")
    
    # 儲存結果
    simp_df, trad_df, combined_df = save_final_results(simplified_results, traditional_results)
    
    # 設定全域變數
    globals()['simplified_chinese_data'] = simp_df
    globals()['traditional_chinese_data'] = trad_df
    globals()['combined_chinese_data'] = combined_df
    
    # 清理臨時檔案（可選）
    print(f"\n🧹 臨時檔案保留狀態:")
    print(f"   ✓ {CHECKPOINT_FILE} - 進度檢查點")
    print(f"   ✓ {PARTIAL_SIMPLIFIED_FILE} - 簡體部分結果")
    print(f"   ✓ {PARTIAL_TRADITIONAL_FILE} - 繁體部分結果")
    print(f"   💡 如需重新開始，請手動刪除這些檔案")
    # os.remove(CHECKPOINT_FILE)
    # os.remove(PARTIAL_SIMPLIFIED_FILE)
    # os.remove(PARTIAL_TRADITIONAL_FILE)
    
    print(f"\n🎉 簡體轉繁體轉換完成！")
    print(f"📋 可用變數: simplified_chinese_data, traditional_chinese_data, combined_chinese_data")
    
except FileNotFoundError:
    print(f"❌ 找不到資料集: {data_path}")
    print(f"💡 請確認檔案路徑是否正確")
except Exception as e:
    print(f"❌ 發生錯誤: {e}")
    import traceback
    traceback.print_exc()

print("="*60)


🚀 啟動簡體轉繁體轉換系統...
📂 載入資料集:  LLM_Aug_102925.csv


📥 讀取資料集: 100%|██████████| 1/1 [00:00<00:00,  3.76it/s]


✅ 成功載入資料集，共 102925 筆記錄
📋 欄位: ['text', 'total_score', 'mainland_terms', 'text_length', 'source_type', 'fragment_length', 'fragment_index', 'fragment_start', 'fragment_end']

🚀 開始非同步批次轉換
🤖 使用模型: gemma-3-27b-it
📦 批次大小: 10 筆/批次
⏱️  預估批次數: 10293
📊 處理資料集：102925 筆
📋 總批次數: 10293
📋 待處理資料: 102925 筆
📋 已完成資料: 0 筆


🔄 批次 699/10293 | 處理中:   7%|▋         | 6980/102925 [47:52<15:21:48,  1.73筆/s, 完成率=6.8%, 已轉換=6980]

⚠️  API 錯誤: 502, message='Attempt to decode JSON with unexpected mimetype: text/plain; charset=utf-8', url='https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions'


...

⚠️  API 錯誤: [Errno 54] Connection reset by peer
⚠️  API 錯誤: [Errno 54] Connection reset by peer
⚠️  API 錯誤: [Errno 54] Connection reset by peer
⚠️  API 錯誤: [Errno 54] Connection reset by peer
⚠️  API 錯誤: [Errno 54] Connection reset by peer
⚠️  API 錯誤: [Errno 54] Connection reset by peer
⚠️  API 錯誤: [Errno 54] Connection reset by peer
⚠️  API 錯誤: [Errno 54] Connection reset by peer
⚠️  API 錯誤: [Errno 54] Connection reset by peer
⚠️  API 錯誤: [Errno 54] Connection reset by peer


🔄 批次 10293/10293 | 處理中: 100%|██████████| 102925/102925 [17:24:35<00:00,  1.64筆/s, 完成率=100.0%, 已轉換=102925]




📊 轉換結果統計:
  📝 簡體中文資料: 102925 筆
  📝 繁體中文資料: 102925 筆
  📈 總資料量: 205850 筆

🔄 轉換範例:
  1.
     簡體: 受伤躺在病床上的沙热古丽·沙依塔这些天来一直在寻找送她到医院的出租车驾驶员，她说...
     繁體: 受傷躺在病床上的沙熱古麗·沙依塔這些天來一直在尋找送她到醫院的計程車駕駛員，她說...
  2.
     簡體: 这组数据给我们老师提了个醒，以后对于咱自己的健康真得长点儿心，不能再忽视大意了。
     繁體: 這組數據給我們老師提個醒，以後對於自己的健康真得長點心，不能再忽視大意了。
  3.
     簡體: 超级划算做工很好卡纸很厚看着很有档次的具体有多少张我没数不过挺多的但不是书的形式...
     繁體: 超級划算做工很好卡紙很厚看起來很有檔次的具體有多少張我沒數不過挺多的但不是書的形...

💾 正在儲存檔案...


📝 儲存簡體中文資料: 100%|██████████| 2/2 [00:00<00:00,  3.78it/s]

📝 儲存合併資料集: 100%|██████████| 1/1 [00:00<00:00,  1.26it/s]


✅ 儲存完成:
  📄 簡體中文資料: simplified_chinese_data_20251029_111151.csv (102925 筆)
  📄 繁體中文資料: traditional_chinese_data_20251029_111151.csv (102925 筆)
  📄 合併資料集: combined_chinese_data_20251029_111151.csv (205850 筆)
  📋 JSON格式也已儲存

🧹 臨時檔案保留狀態:
   ✓ conversion_checkpoint.json - 進度檢查點
   ✓ simplified_partial.csv - 簡體部分結果
   ✓ traditional_partial.csv - 繁體部分結果
   💡 如需重新開始，請手動刪除這些檔案

🎉 簡體轉繁體轉換完成！
📋 可用變數: simplified_chinese_data, traditional_chinese_data, combined_chinese_data


# 🔄 字形統一處理

為了防止模型「作弊」（僅根據簡繁字體差異分類），我們需要：
1. **簡體資料**：字形轉為繁體，但詞彙保持大陸用語（如「計算機」）
2. **繁體資料**：已經是台灣繁體用語（如「電腦」）

這樣模型必須學習**詞彙和語法差異**，而非字形差異。

In [4]:
# 字形統一轉換函數
def convert_simplified_to_traditional_form_only(text):
    """
    僅將簡體字形轉換為繁體字形，不改變詞彙
    例如："计算机" -> "計算機"（而非 "電腦"）
    使用 OpenCC 的 s2t（簡體到繁體）轉換
    """
    try:
        from opencc import OpenCC
        cc = OpenCC('s2t')  # 簡體到繁體（僅字形）
        return cc.convert(text)
    except ImportError:
        print("⚠️  OpenCC 未安裝，使用基本轉換")
        # 備用方案：返回原文
        return text
    except Exception as e:
        print(f"⚠️  轉換錯誤: {e}")
        return text

# 測試字形轉換
print("📝 字形轉換測試:")
test_cases = [
    "我在用计算机处理数据",
    "这个软件很好用",
    "网络连接很快",
    "出租车司机很友好"
]

for text in test_cases:
    converted = convert_simplified_to_traditional_form_only(text)
    print(f"  簡體: {text}")
    print(f"  繁體: {converted}")
    print()

📝 字形轉換測試:
  簡體: 我在用计算机处理数据
  繁體: 我在用計算機處理數據

  簡體: 这个软件很好用
  繁體: 這個軟件很好用

  簡體: 网络连接很快
  繁體: 網絡連接很快

  簡體: 出租车司机很友好
  繁體: 出租車司機很友好



# 📊 準備對比學習資料集

將轉換後的資料準備成單句對比學習格式，適用於 `contractive_finetune.ipynb`

In [5]:
def prepare_contrastive_learning_data(simplified_df, traditional_df, output_dir="data_cache_v4", 
                                      unify_character_form=True):
    """
    準備單句對比學習資料集
    參考 contractive_finetune.ipynb 第 1582-1607 行的實作
    
    Args:
        simplified_df: 簡體中文資料 DataFrame
        traditional_df: 繁體中文資料 DataFrame
        output_dir: 輸出目錄
        unify_character_form: 是否統一字形（將簡體字轉為繁體字，但保持詞彙不變）
    
    Returns:
        train_df, val_df: 訓練集和驗證集
    """
    import pandas as pd
    from sklearn.model_selection import train_test_split
    import os
    from pathlib import Path
    
    print("="*60)
    print("🎯 準備對比學習資料集（單句格式）")
    if unify_character_form:
        print("🔄 啟用字形統一：將簡體字轉為繁體字，但保持詞彙差異")
    print("="*60)
    
    # 1. 確保資料有正確的標籤
    simp_df_copy = simplified_df.copy()
    trad_df_copy = traditional_df.copy()
    
    # 2. 字形統一處理（如果啟用）
    if unify_character_form:
        print(f"\n🔄 正在將簡體資料的字形統一為繁體...")
        from opencc import OpenCC
        cc = OpenCC('s2t')  # 簡體到繁體（僅字形轉換）
        
        # 使用 tqdm 顯示進度
        tqdm.pandas(desc="字形轉換")
        simp_df_copy['text'] = simp_df_copy['text'].progress_apply(
            lambda x: cc.convert(str(x)) if pd.notna(x) else x
        )
        
        print(f"✅ 字形統一完成")
        
        # 顯示轉換範例
        print(f"\n📝 字形統一範例:")
        for i in range(min(3, len(simplified_df))):
            original = simplified_df.iloc[i]['text'][:60]
            converted = simp_df_copy.iloc[i]['text'][:60]
            print(f"  {i+1}.")
            print(f"     原始簡體: {original}...")
            print(f"     繁體字形: {converted}...")
            print()
    
    # 3. 設定標籤：0 = 簡體（大陸用語），1 = 繁體（台灣用語）
    simp_df_copy['label'] = 0
    trad_df_copy['label'] = 1
    
    # 4. 合併資料集
    clean_df = pd.concat([simp_df_copy, trad_df_copy], ignore_index=True)
    
    print(f"\n📊 資料集統計:")
    print(f"  簡體中文 (Label 0 - 大陸用語): {len(simp_df_copy):,} 筆")
    print(f"  繁體中文 (Label 1 - 台灣用語): {len(trad_df_copy):,} 筆")
    print(f"  總計: {len(clean_df):,} 筆")
    
    if unify_character_form:
        print(f"\n✅ 字形已統一為繁體，模型需學習詞彙和語法差異")
        print(f"   例如: 計算機 (大陸) vs 電腦 (台灣)")
    
    print(f"\n📋 標籤分布:")
    print(clean_df['label'].value_counts().sort_index())
    
    # 5. 檢查必要欄位
    required_cols = ['text', 'label']
    missing_cols = [col for col in required_cols if col not in clean_df.columns]
    
    if missing_cols:
        raise ValueError(f"缺少必要欄位: {missing_cols}")
    
    # 6. 清理資料
    print(f"\n🧹 清理資料...")
    initial_count = len(clean_df)
    
    # 移除空白文本
    clean_df = clean_df[clean_df['text'].notna()]
    clean_df = clean_df[clean_df['text'].str.strip() != '']
    
    # 移除重複文本（保留第一個）
    clean_df = clean_df.drop_duplicates(subset=['text'], keep='first')
    
    print(f"  移除空白/重複後: {len(clean_df):,} 筆 (移除 {initial_count - len(clean_df):,} 筆)")
    
    # 7. 分層分割（stratified split）
    print(f"\n✂️  分割訓練集和驗證集...")
    train_df, val_df = train_test_split(
        clean_df, 
        test_size=0.1, 
        random_state=42,
        stratify=clean_df['label']
    )
    
    print(f"\n📊 分割結果:")
    print(f"  訓練集: {len(train_df):,} 筆")
    print(f"  驗證集: {len(val_df):,} 筆")
    
    print(f"\n📋 訓練集標籤分布:")
    print(train_df['label'].value_counts().sort_index())
    
    print(f"\n📋 驗證集標籤分布:")
    print(val_df['label'].value_counts().sort_index())
    
    # 8. 儲存資料集
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    train_path = Path(output_dir) / "train_contrastive.parquet"
    val_path = Path(output_dir) / "val_contrastive.parquet"
    full_path = Path(output_dir) / "cleaned_dataset_contrastive.parquet"
    
    print(f"\n💾 儲存資料集...")
    with tqdm(total=3, desc="儲存進度") as pbar:
        train_df.to_parquet(train_path, index=False, compression='zstd')
        pbar.update(1)
        
        val_df.to_parquet(val_path, index=False, compression='zstd')
        pbar.update(1)
        
        clean_df.to_parquet(full_path, index=False, compression='zstd')
        pbar.update(1)
    
    print(f"\n✅ 儲存完成:")
    print(f"  📄 訓練集: {train_path}")
    print(f"  📄 驗證集: {val_path}")
    print(f"  📄 完整資料: {full_path}")
    
    if unify_character_form:
        print(f"\n🎯 對比學習資料集準備完成（字形已統一）！")
        print(f"   模型將學習: 詞彙差異 + 語法差異，而非字形差異")
    else:
        print(f"\n🎯 對比學習資料集準備完成！")
    
    print(f"📝 可直接用於 contractive_finetune.ipynb 訓練")
    print("="*60)
    
    return train_df, val_df


# 執行資料準備
if 'simplified_chinese_data' in globals() and 'traditional_chinese_data' in globals():
    print("\n開始準備對比學習資料集...")
    
    # 啟用字形統一
    contrastive_train_df, contrastive_val_df = prepare_contrastive_learning_data(
        simplified_chinese_data,
        traditional_chinese_data,
        output_dir="data_cache_v4",
        unify_character_form=True  # 🔑 關鍵參數：統一字形
    )
    
    # 設定全域變數
    globals()['contrastive_train_df'] = contrastive_train_df
    globals()['contrastive_val_df'] = contrastive_val_df
    
    print("\n✅ 資料已準備完成！可以開始對比學習訓練")
    print("📋 可用變數: contrastive_train_df, contrastive_val_df")
    
else:
    print("⚠️  請先執行上一個 cell 生成 simplified_chinese_data 和 traditional_chinese_data")


開始準備對比學習資料集...
🎯 準備對比學習資料集（單句格式）
🔄 啟用字形統一：將簡體字轉為繁體字，但保持詞彙差異

🔄 正在將簡體資料的字形統一為繁體...


字形轉換: 100%|██████████| 102925/102925 [00:03<00:00, 33540.33it/s]



✅ 字形統一完成

📝 字形統一範例:
  1.
     原始簡體: 受伤躺在病床上的沙热古丽·沙依塔这些天来一直在寻找送她到医院的出租车驾驶员，她说：“这个汉族小伙子是我的救命恩人，他在我...
     繁體字形: 受傷躺在病牀上的沙熱古麗·沙依塔這些天來一直在尋找送她到醫院的出租車駕駛員，她說：“這個漢族小夥子是我的救命恩人，他在我...

  2.
     原始簡體: 这组数据给我们老师提了个醒，以后对于咱自己的健康真得长点儿心，不能再忽视大意了。...
     繁體字形: 這組數據給我們老師提了個醒，以後對於咱自己的健康真得長點兒心，不能再忽視大意了。...

  3.
     原始簡體: 超级划算做工很好卡纸很厚看着很有档次的具体有多少张我没数不过挺多的但不是书的形式每个手工都是单独的一张卡纸颜色非常好没有...
     繁體字形: 超級划算做工很好卡紙很厚看着很有檔次的具體有多少張我沒數不過挺多的但不是書的形式每個手工都是單獨的一張卡紙顏色非常好沒有...


📊 資料集統計:
  簡體中文 (Label 0 - 大陸用語): 102,925 筆
  繁體中文 (Label 1 - 台灣用語): 102,925 筆
  總計: 205,850 筆

✅ 字形已統一為繁體，模型需學習詞彙和語法差異
   例如: 計算機 (大陸) vs 電腦 (台灣)

📋 標籤分布:
label
0    102925
1    102925
Name: count, dtype: int64

🧹 清理資料...
  移除空白/重複後: 196,031 筆 (移除 9,819 筆)

✂️  分割訓練集和驗證集...

📊 分割結果:
  訓練集: 176,427 筆
  驗證集: 19,604 筆

📋 訓練集標籤分布:
label
0    90778
1    85649
Name: count, dtype: int64

📋 驗證集標籤分布:
label
0    10087
1     9517
Name: count, dtype: int64

💾 儲存資料集...


儲存進度: 100%|██████████| 3/3 [00:00<00:00,  8.46it/s]


✅ 儲存完成:
  📄 訓練集: data_cache_v4/train_contrastive.parquet
  📄 驗證集: data_cache_v4/val_contrastive.parquet
  📄 完整資料: data_cache_v4/cleaned_dataset_contrastive.parquet

🎯 對比學習資料集準備完成（字形已統一）！
   模型將學習: 詞彙差異 + 語法差異，而非字形差異
📝 可直接用於 contractive_finetune.ipynb 訓練

✅ 資料已準備完成！可以開始對比學習訓練
📋 可用變數: contrastive_train_df, contrastive_val_df


# 🚀 如何使用對比學習訓練

## 🔑 關鍵改進：字形統一

為了防止模型「作弊」（僅根據簡繁字體判斷），我們採用**字形統一**策略：

### 資料處理方式

1. **簡體資料（Label 0 - 大陸用語）**
   - ✅ 字形：簡體 → 繁體（如：计算机 → 計算機）
   - ✅ 詞彙：保持大陸用語（如：計算機、軟件、網絡）
   
2. **繁體資料（Label 1 - 台灣用語）**
   - ✅ 字形：繁體（保持不變）
   - ✅ 詞彙：台灣用語（如：電腦、軟體、網路）

### 學習目標

模型**無法**依賴字形差異，必須學習：
- ✅ **詞彙差異**：計算機 vs 電腦、軟件 vs 軟體
- ✅ **語法習慣**：挺好的 vs 很好、咋樣 vs 怎麼樣
- ✅ **表達方式**：大陸口語 vs 台灣口語

### 範例對比

```
簡體資料（字形已轉繁體，詞彙保持大陸用語）：
"這個計算機的軟件運行得挺快的"
      ↑         ↑        ↑
   大陸用語   大陸用語   大陸語法

繁體資料（台灣用語）：
"這個電腦的軟體運行得很快"
      ↑      ↑      ↑
   台灣用語  台灣用語  台灣語法
```

---

## 資料格式說明

準備好的資料集包含以下欄位：
- `text`: 中文文本（**字形統一為繁體**）
- `label`: 標籤 (0=大陸用語, 1=台灣用語)
- 其他原始欄位（如 `text_length`, `language_type` 等）

## 訓練步驟

### 1. 使用現有的 contractive_finetune.ipynb

資料已儲存在 `data_cache_v4/` 目錄：
- `train_contrastive.parquet` - 訓練集
- `val_contrastive.parquet` - 驗證集
- `cleaned_dataset_contrastive.parquet` - 完整資料集

直接在 `contractive_finetune.ipynb` 中修改資料路徑：

```python
# 載入資料（字形已統一）
clean_df = pd.read_parquet("data_cache_v4/cleaned_dataset_contrastive.parquet")
```

### 2. 對比學習的優勢

- ✅ **學習真實差異**：專注於詞彙和語法，而非字形
- ✅ **更強的泛化能力**：不會被字形混淆
- ✅ **更好的表徵空間**：同類用語在向量空間中更接近
- ✅ **實際應用價值**：能區分真實的語言使用差異

### 3. 訓練配置建議

參考 `contractive_finetune.ipynb` 的設定：

```python
# 模型
BASE_MODEL = "ckiplab/bert-base-chinese"

# 訓練參數
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
EPOCHS = 3
TEMPERATURE = 0.07  # 對比學習溫度參數

# 資料增強
MASK_PROB = 0.12
MAX_MASK_TOKENS = 3
```

### 4. 預期效果

使用字形統一後，模型應該能夠：
1. ✅ 正確識別大陸用語和台灣用語的差異
2. ✅ 理解語法習慣的不同
3. ✅ 在混合字形的文本中也能準確分類
4. ✅ 具有更強的實際應用價值

### 5. 評估指標

- **準確率 (Accuracy)**：整體分類正確率
- **F1-Score**：平衡精確率和召回率
- **混淆矩陣**：查看誤分類情況
- **對比損失 (Contrastive Loss)**：訓練過程中的主要損失
- **相似度間隔 (Similarity Gap)**：同類 vs 異類的相似度差距

---

**下一步**：執行 `contractive_finetune.ipynb` 開始訓練！